In [11]:
# Imports for Deep Learning
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.vgg19 import (
    VGG19, 
    preprocess_input, 
    decode_predictions
)
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import numpy as np

# Ensure consistency across runs
from numpy.random import seed
import random
seed(2)
import tensorflow

# Imports to view data
import cv2
from glob import glob

# Metrics
from sklearn.metrics import classification_report, confusion_matrix

# Visualization
from keras.utils import print_summary
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

# Utils
from pathlib import Path
import pandas as pd
import numpy as np
from os import getenv
import time
import itertools

# Image Preprocessing
from skimage.filters import sobel, scharr

# Global Variable Setup

In [30]:
TRAIN_DIR = 'C:/Bootcamp/Homework/proj3-team04/Model Training/asl_alphabet_train/asl_alphabet_train'
TEST_DIR = 'C:/Bootcamp/Homework/proj3-team04/Model Training/asl_alphabet_test/asl_alphabet_test'
CUSTOM_TEST_DIR = '../input/asl-alphabet-test/asl-alphabet-test'
CLASSES = [folder[len(TRAIN_DIR) + 1:] for folder in glob(TRAIN_DIR + '/*')]
CLASSES.sort()

TARGET_SIZE = (64, 64)
TARGET_DIMS = (64, 64, 3) # add channel for RGB
N_CLASSES = 29
VALIDATION_SPLIT = 0.1
BATCH_SIZE = 64

# Model saving for easier local iterations
MODEL_DIR = 'C:/Bootcamp/Homework/proj3-team04/Model Training/'
MODEL_PATH = MODEL_DIR + '/cnn-model.h5'
MODEL_WEIGHTS_PATH = MODEL_DIR + '/cnn-model.weights.h5'
MODEL_SAVE_TO_DISK = 'C:/Bootcamp/Homework/proj3-team04/Model Training/'

In [31]:
def preprocess_image(image):
    '''Function that will be implied on each input. The function
    will run after the image is resized and augmented.
    The function should take one argument: one image (Numpy tensor
    with rank 3), and should output a Numpy tensor with the same
    shape.'''
    sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=5)
    return sobely

def make_generator(options):
    '''Creates two generators for dividing and preprocessing data.'''
    validation_split = options.get('validation_split', 0.0)
    preprocessor = options.get('preprocessor', None)
    data_dir = options.get('data_dir', TRAIN_DIR)

    augmentor_options = {
        'samplewise_center': True,
        'samplewise_std_normalization': True,
    }
    if validation_split is not None:
        augmentor_options['validation_split'] = validation_split
    
    if preprocessor is not None:
        augmentor_options['preprocessing_function'] = preprocessor
    
    flow_options = {
        'target_size': TARGET_SIZE,
        'batch_size': BATCH_SIZE,
        'shuffle': options.get('shuffle', None),
        'subset': options.get('subset', None),
    }

    data_augmentor = ImageDataGenerator(**augmentor_options)
    return data_augmentor.flow_from_directory(data_dir, **flow_options)

In [32]:
def load_model_from_disk():
    '''A convenience method for re-running certain parts of the
    analysis locally without refitting all the data.'''
    model_file = Path(MODEL_PATH)
    model_weights_file = Path(MODEL_WEIGHTS_PATH)
                      
    if model_file.is_file() and model_weights_file.is_file():
        print('Retrieving model from disk...')
        model = load_model(model_file.__str__())
                      
        print('Loading CNN model weights from disk...')
        model.load_weights(model_weights_file)
        return model
    
    return None

CNN_MODEL = load_model_from_disk()
REPROCESS_MODEL = (CNN_MODEL is None)

print('Need to reprocess? {}'.format(REPROCESS_MODEL))

Need to reprocess? True


In [34]:
def build_model(save=False):
    print('Building model afresh...')
    
    model = Sequential()
    
    model.add(Conv2D(64, kernel_size=4, strides=1, activation='relu', input_shape=TARGET_DIMS))
    model.add(Conv2D(64, kernel_size=4, strides=2, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Conv2D(128, kernel_size=4, strides=1, activation='relu'))
    model.add(Conv2D(128, kernel_size=4, strides=2, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Conv2D(256, kernel_size=4, strides=1, activation='relu'))
    model.add(Conv2D(256, kernel_size=4, strides=2, activation='relu'))
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(N_CLASSES, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    if save: model.save(MODEL_PATH)
        
    return model

# if REPROCESS_MODEL:
#     CNN_MODEL = build_model(save=MODEL_SAVE_TO_DISK)

# print_summary(CNN_MODEL)

In [35]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 61, 61, 64)        3136      
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 29, 29, 64)        65600     
_________________________________________________________________
dropout_9 (Dropout)          (None, 29, 29, 64)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 26, 26, 128)       131200    
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 12, 12, 128)       262272    
_________________________________________________________________
dropout_10 (Dropout)         (None, 12, 12, 128)       0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 9, 9, 256)        

In [36]:
def make_generator_for(subset):
    '''Create a generator for the training or validation set.'''
    generator_options = dict(
        validation_split=VALIDATION_SPLIT,
        shuffle=True,
        subset=subset,
        preprocessor=preprocess_image,
    )
    return make_generator(generator_options)


def fit_model(model, train_generator, val_generator, save=False):
    '''Fit the model with the training and validation generators.'''    
    history = model.fit_generator(train_generator, epochs=5, validation_data=val_generator)
    
    if save: model.save_weights(MODEL_WEIGHTS_PATH)
    
    return history


CNN_TRAIN_GENERATOR = make_generator_for('training')
CNN_VAL_GENERATOR = make_generator_for('validation')

HISTORY = None
if REPROCESS_MODEL:
    start_time = time.time()
    HISTORY = fit_model(CNN_MODEL, CNN_TRAIN_GENERATOR, CNN_VAL_GENERATOR, save=MODEL_SAVE_TO_DISK)
    print('Fitting the model took ~{:.0f} second(s).'.format(time.time() - start_time))


columns=['Dimension 1', 'Dimension 2', 'Dimension 3', 'Dimension 4']
pd.DataFrame(data=[x.shape for x in CNN_MODEL.weights], columns=columns)

Found 78300 images belonging to 29 classes.
Found 8700 images belonging to 29 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
1224/1224 [==============================] - 1880s 2s/step - loss: 1.2814 - accuracy: 0.5890 - val_loss: 0.8117 - val_accuracy: 0.7220
Epoch 2/5
1224/1224 [==============================] - 1736s 1s/step - loss: 0.3837 - accuracy: 0.8681 - val_loss: 0.6793 - val_accuracy: 0.8141
Epoch 3/5
1224/1224 [==============================] - 1703s 1s/step - loss: 0.2551 - accuracy: 0.9130 - val_loss: 0.6580 - val_accuracy: 0.8017
Epoch 4/5
1224/1224 [==============================] - 1586s 1s/step - loss: 0.1998 - accuracy: 0.9326 - val_loss: 0.6639 - val_accuracy: 0.8257
Epoch 5/5
1224/1224 [==============================] - 1569s 1s/step - loss: 0.1770 - accuracy: 0.9427 - val_loss: 0.5358 - val_accuracy: 0.8494
Fitting the model took ~8483 second(s).


,Dimension 1,Dimension 2,Dimension 3,Dimension 4
0,4,4.0,3.0,64.0
1,64,NaN,NaN,NaN
2,4,4.0,64.0,64.0
3,64,NaN,NaN,NaN
4,4,4.0,64.0,128.0
5,128,NaN,NaN,NaN
6,4,4.0,128.0,128.0
7,128,NaN,NaN,NaN
8,4,4.0,128.0,256.0
9,256,NaN,NaN,NaN


In [37]:
model.save('test_model_1.h5', save_format='h5')

In [38]:
def evaluate_model(generator):
    start_time = time.time()
    evaluations = CNN_MODEL.evaluate_generator(generator)
    for i in range(len(CNN_MODEL.metrics_names)):
        print("{}: {:.2f}%".format(
            CNN_MODEL.metrics_names[i], evaluations[i] * 100))
    print('Took {:.0f} seconds to evaluate this set.'.format(
        time.time() - start_time))

    start_time = time.time()
    predictions = CNN_MODEL.predict_generator(generator)
    print('Took {:.0f} seconds to get predictions on this set.'.format(
        time.time() - start_time))

    y_pred = np.argmax(predictions, axis=1)
    y_true = generator.classes
    return dict(y_pred=y_pred, y_true=y_true)


def evaluate_validation_dataset():
    gen_options = dict(
        validation_split=0.1,
        data_dir=TRAIN_DIR,
        shuffle=False,
        subset='validation',
        preprocessor=preprocess_image,
    )
    val_gen = make_generator(gen_options)
    return evaluate_model(val_gen)


def evaluate_test_dataset():
    gen_options = dict(
        validation_split=0.0,
        data_dir=TEST_DIR,
        shuffle=False,
        preprocessor=preprocess_image,
    )
    test_gen = make_generator(gen_options)
    return evaluate_model(test_gen)

In [39]:
CNN_VALIDATION_SET_EVAL = evaluate_validation_dataset()

Found 8700 images belonging to 29 classes.
Instructions for updating:
Please use Model.evaluate, which supports generators.
loss: 53.58%
accuracy: 84.94%
Took 42 seconds to evaluate this set.
Instructions for updating:
Please use Model.predict, which supports generators.
Took 45 seconds to get predictions on this set.


In [40]:
print(classification_report(**CNN_VALIDATION_SET_EVAL, target_names=CLASSES))

              precision    recall  f1-score   support

           A       0.74      0.87      0.80       300
           B       0.89      0.99      0.94       300
           C       0.98      0.98      0.98       300
           D       0.97      0.92      0.94       300
           E       0.63      0.86      0.73       300
           F       1.00      0.98      0.99       300
           G       0.99      0.88      0.93       300
           H       0.89      0.99      0.94       300
           I       0.84      0.66      0.74       300
           J       1.00      0.99      1.00       300
           K       0.91      0.91      0.91       300
           L       0.90      0.99      0.94       300
           M       0.85      0.79      0.82       300
           N       0.76      0.61      0.68       300
           O       1.00      0.71      0.83       300
           P       0.99      0.97      0.98       300
           Q       0.97      0.99      0.98       300
           R       0.88    

In [41]:
print(CLASSES)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']
